<a href="https://colab.research.google.com/github/BiniyamAschalew/BiniyamAschalew/blob/main/IE343sub7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('content')
input_dir = "content/MyDrive/IE343/"

Drive already mounted at content; to attempt to forcibly remount, call drive.mount("content", force_remount=True).


In [ ]:
from google.colab import files

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  HistGradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, KFold, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import re
import optuna
from optuna.integration import XGBoostPruningCallback

In [ ]:
train_data = pd.read_csv(input_dir + "train.csv")
test_data = pd.read_csv(input_dir + "test.csv")

In [ ]:
# preprocessing

In [ ]:
tr = train_data.copy()
tst = test_data.copy()

In [ ]:
# change the columns with object dtype to numbers by encoding them
le = LabelEncoder()
label_columns = list(tr.select_dtypes('object'))
for col in label_columns:
  le.fit(tr[col])
  tr[col] = le.transform(tr[col])
  tst[col] = le.transform(tst[col])

In [ ]:
min_year = tr['transaction_year'].min()
tr['cpi'] = (tr['transaction_year'] - min_year) * 12 + tr['transaction_month']
tr['age'] = tr['transaction_year'] - tr['built_year']

tst['cpi'] = (tst['transaction_year'] - min_year) * 12 + tst['transaction_month']
tst['age'] = tst['transaction_year'] - tst['built_year']

tr.head()
tr.dtypes

index                  int64
apartment_id           int64
city                   int64
dong                   int64
house_area           float64
built_year             int64
floor                  int64
lat                  float64
long                 float64
transaction_year       int64
transaction_month      int64
transaction_day        int64
PRICE                float64
cpi                    int64
age                    int64
dtype: object

In [ ]:
# see the effect at the end

# scale lat and long
# scaler_lat = StandardScaler()
# scaler_long = StandardScaler()

# # tr.loc[tr['dong'] != 113,'lat'] = scaler_lat.fit_transform(tr[tr['dong'] != 113]['lat'])
# scaler_lat.fit(tr[tr['dong'] != 113][['lat']])
# tr.loc[tr['dong'] != 113, 'lat'] = scaler_lat.transform(tr[tr['dong'] != 113][['lat']])
# tst.loc[tst['dong'] != 113, 'lat'] = scaler_lat.transform(tst[tst['dong'] != 113][['lat']])

# scaler_long.fit(tr[tr['dong'] != 113][['long']])
# tr.loc[tr['dong'] != 113, 'long'] = scaler_long.transform(tr[tr['dong'] != 113][['long']])
# tst.loc[tst['dong'] != 113, 'long'] = scaler_long.transform(tst[tst['dong'] != 113][['long']])

# tr.loc[:, 'lat'].mean()

In [ ]:
# Taking care of missing values
# the only missing values exist in busan dong 113
# solution lets assign its values with normal distribution with mean -> average of all lat and long in busan, sd of sd of the others
busan_dong_train = tr[tr['city'] == 0].loc[:, ["dong", "lat", "long"]]
busan_dong_test = tst[tst['city'] == 0].loc[:, ["dong", "lat", "long"]]
busan_dong = pd.concat([busan_dong_train, busan_dong_test])

mean_lat1, mean_long1 = busan_dong[busan_dong['dong'] == 115].mean()[1:]
mean_lat2, mean_long2 = busan_dong[busan_dong['dong'] == 114].mean()[1:]
mean_lat = (mean_lat1 + mean_lat2)/2
mean_long = (mean_long1 + mean_long2)/2
mean_lat, mean_long

std_lat1, std_long1 = busan_dong[busan_dong['dong'] == 115].std()[1:]
std_lat2, std_long2 = busan_dong[busan_dong['dong'] == 114].std()[1:]
std_lat = (std_lat1 + std_lat2)/2
std_long = (std_long1 + std_long2)/2
std_lat, std_long

na_train = 89
na_test = 13


# busan_dong.head()


In [ ]:
# now let's scale lat and long to have standard normal dist


In [ ]:

tst.loc[tst['dong'] == 113,'lat'] = np.random.randn(na_test) * std_lat + mean_lat
tst.loc[tst['dong'] == 113,'long'] = np.random.randn(na_test) * std_long + mean_long

tr.loc[tr['dong'] == 113,'lat'] = np.random.randn(na_train) * std_lat + mean_lat
tr.loc[tr['dong'] == 113,'long'] = np.random.randn(na_train) * std_long + mean_long

In [ ]:
# since the apartment id is random hash name of apartments, lets converted it to also represent the average price of that apartment frenchise

tr['apartment_rank'] = np.zeros(len(tr))
tst['apartment_rank'] = np.zeros(len(tst))


apartment_price = tr.groupby('apartment_id')['PRICE'].agg('mean').sort_values(ascending=False)
apartment_price # I want to sort them and put the sorted
for idx, apartment in enumerate(list(apartment_price.index)):
  tr.loc[tr['apartment_id'] == apartment, 'apartment_rank' ] = idx
  tst.loc[tst['apartment_id'] == apartment, 'apartment_rank' ] = idx

# sns.regplot(x=tr['apartment_rank'], y = tr['PRICE'])

In [ ]:
x_train2 = tr.copy()
x_test2 = tst.copy()

finx_train = x_train2.drop(columns = 'PRICE')
finy_train = x_train2['PRICE']
x_test2

,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,cpi,age,apartment_rank
0,329690,0,0,197,101.647190,1993,3,35.149929,129.006071,2023,1,5,61,30,3180.0
1,329691,0,0,197,91.511175,1993,12,35.149929,129.006071,2023,2,1,62,30,3180.0
2,329692,0,0,197,125.865988,1993,2,35.149929,129.006071,2023,2,0,62,30,3180.0
3,329693,0,0,197,101.647190,1993,8,35.149929,129.006071,2023,2,2,62,30,3180.0
4,329694,0,0,197,101.647190,1993,13,35.149929,129.006071,2023,3,5,63,30,3180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85092,414782,4419,1,37,100.821957,2014,8,37.452039,127.070842,2023,10,5,70,9,396.0
85093,414783,4419,1,37,101.431912,2014,11,37.452039,127.070842,2023,10,5,70,9,396.0
85094,414784,4419,1,37,121.201627,2014,12,37.452039,127.070842,2023,11,1,71,9,396.0
85095,414785,4419,1,37,137.192013,2014,3,37.452039,127.070842,2023,11,4,71,9,396.0


In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression', max_depth=7, learning_rate=0.0007953, n_estimators = 8829, min_child_samples = 85, subsample=0.5223234814923825)
model_lgb.fit(finx_train, finy_train)



LGBMRegressor(learning_rate=0.0007953, max_depth=7, min_child_samples=85,
              n_estimators=8829, objective='regression',
              subsample=0.5223234814923825)

In [173]:
# new one
model_lgb2 = lgb.LGBMRegressor( max_depth=7, learning_rate=0.001, n_estimators = 10000, min_child_samples = 82, subsample=0.5223234814923825)
model_lgb2.fit(finx_train, finy_train)

y_pred2 = model_lgb2.predict(x_test2)

output = pd.concat([x_test2['index'], pd.DataFrame(y_pred2, columns = ["PRICE"])], axis = 1)

output.to_csv('subm6.csv', index=False)
files.download('subm6.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [175]:
model_lgb3 = lgb.LGBMRegressor( n_estimators = 10000)
model_lgb3.fit(finx_train, finy_train)

y_pred3 = model_lgb3.predict(x_test2)

output = pd.concat([x_test2['index'], pd.DataFrame(y_pred3, columns = ["PRICE"])], axis = 1)

output.to_csv('subm7.csv', index=False)  
files.download('subm7.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [177]:
model_lgb4 = lgb.LGBMRegressor( )
model_lgb4.fit(finx_train, finy_train)

LGBMRegressor()

In [176]:
model_lgb4 = lgb.LGBMRegressor()
model_lgb4.fit(finx_train, finy_train)

y_pred3 = model_lgb4.predict(x_test2)

output = pd.concat([x_test2['index'], pd.DataFrame(y_pred3, columns = ["PRICE"])], axis = 1)

output.to_csv('subm8.csv', index=False)
files.download('subm8.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [178]:
model_lgb5 = lgb.LGBMRegressor(n_estimators=15000)
model_lgb5.fit(finx_train, finy_train)

y_pred4 = model_lgb5.predict(x_test2)

output = pd.concat([x_test2['index'], pd.DataFrame(y_pred4, columns = ["PRICE"])], axis = 1)

output.to_csv('subm9.csv', index=False)
files.download('subm9.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [172]:
finx_train # normalizing lat and long had almost 0 effect

,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,cpi,age,apartment_rank
0,0,0,0,197,125.865988,1993,5,35.149929,129.006071,2021,7,0,43,28,3180.0
1,1,0,0,197,101.647190,1993,12,35.149929,129.006071,2021,10,1,46,28,3180.0
2,2,0,0,197,91.511175,1993,6,35.149929,129.006071,2020,3,5,27,27,3180.0
3,3,0,0,197,101.647190,1993,13,35.149929,129.006071,2020,5,0,29,27,3180.0
4,4,0,0,197,101.647190,1993,4,35.149929,129.006071,2022,6,4,54,29,3180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329685,329685,4419,1,37,101.431912,2014,4,37.452039,127.070842,2022,5,5,53,8,396.0
329686,329686,4419,1,37,101.431912,2014,14,37.452039,127.070842,2021,10,1,46,7,396.0
329687,329687,4419,1,37,71.687641,2014,2,37.452039,127.070842,2022,11,4,59,8,396.0
329688,329688,4419,1,37,137.192013,2014,18,37.452039,127.070842,2020,9,4,33,6,396.0


In [169]:
model_lgb

y_pred = model_lgb.predict(x_test2)

output = pd.concat([tst['index'], pd.DataFrame(y_pred, columns = ["PRICE"])], axis = 1)

In [ ]:
# {'max_depth': 7, 'learning_rate': 0.0007953098131062764, 'n_estimators': 8829, 'min_child_samples': 85, 'subsample': 0.5223234814923825} trial 14  value: 44711.82480833971.
# {'max_depth': 11, 'learning_rate': 0.005252427629146827, 'n_estimators': 8015, 'min_child_samples': 54, 'subsample': 0.8422371309477078} trial 2 with value: 47773.23330978951.
# {'max_depth': 7, 'learning_rate': 0.001023558187447401, 'n_estimators': 9524, 'min_child_samples': 82, 'subsample': 0.4785940319002228}. Best is trial 21 with value: 44231.935269501104

In [165]:
# subm4 was with the second hyperparameter above, it got 61k 
model_lgb

LGBMRegressor(learning_rate=0.0007953, max_depth=7, min_child_samples=85,
              n_estimators=8829, objective='regression',
              subsample=0.5223234814923825)

In [170]:
output.to_csv('subm5.csv', index=False)
files.download('subm5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# lets try it without normalizing the lat and long
# subm 5 is without normalizing lat and long

In [167]:
x_test2

,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,cpi,age,apartment_rank
0,329690,0,0,197,101.647190,1993,3,35.149929,129.006071,2023,1,5,61,30,3180.0
1,329691,0,0,197,91.511175,1993,12,35.149929,129.006071,2023,2,1,62,30,3180.0
2,329692,0,0,197,125.865988,1993,2,35.149929,129.006071,2023,2,0,62,30,3180.0
3,329693,0,0,197,101.647190,1993,8,35.149929,129.006071,2023,2,2,62,30,3180.0
4,329694,0,0,197,101.647190,1993,13,35.149929,129.006071,2023,3,5,63,30,3180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85092,414782,4419,1,37,100.821957,2014,8,37.452039,127.070842,2023,10,5,70,9,396.0
85093,414783,4419,1,37,101.431912,2014,11,37.452039,127.070842,2023,10,5,70,9,396.0
85094,414784,4419,1,37,121.201627,2014,12,37.452039,127.070842,2023,11,1,71,9,396.0
85095,414785,4419,1,37,137.192013,2014,3,37.452039,127.070842,2023,11,4,71,9,396.0
